## Combined models
1) rf-minimal
2) rf-general
3) rf-lexical

In [40]:
import os
import pickle

from collections import defaultdict
import numpy as np
import pandas as pd 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler

In [41]:
def classification_type(type):
    '''
    Convert classification type into values:
        1) Benign = 0
        2) Defacement = 1
        3) Phishing = 2
        4) Malware = 3    
    '''
    if type == "benign":
        return 0
    elif type == "defacement":
        return 1
    elif type == "phishing":
        return 2
    elif type == "malware":
        return 3
    else:
        print(f"Unable to find proper type: {type}")


def calc_FNR_accuracy(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    for label_class in range(4):
        FN = sum(conf_matrix[label_class][i] for i in range(len(conf_matrix)) if i != label_class)  
        
        TP = conf_matrix[label_class][label_class]  
        
        TN = np.sum(np.delete(np.delete(conf_matrix, label_class, axis=0), label_class, axis=1))
        
        accuracy = (TP + TN) / np.sum(conf_matrix)
        print("Accuracy for class", label_class, ":", accuracy)

        FNR = FN / (FN + TP) if (FN + TP) > 0 else -1
        print("FNR for class", label_class, ":", FNR)

In [42]:
# Load models
model_files = [file for file in os.listdir(os.getcwd()) if file.endswith(".pkl")]
models = []
for file in model_files:
    with open(file, "rb") as file:
        models.append((file.name, pickle.load(file)))

In [43]:
# Data for training ensemble model
training_dataset = {'rf-minimal.pkl': 'rf-minimal-features.csv',
                    'rf-general.pkl': 'rf-general-features.csv',
                    'rf-lexical.pkl': 'rf-lexical-features.csv'}

df = pd.read_csv('../datasets/malicious_phish.csv')
y = df.iloc[:, 1]
y = y.apply(classification_type)
y = y.values.ravel()

In [44]:
testing_ensemble_X = {} # test size 0.2 values <model_name: (X_test, scaler)>
testing_ensemble_y = None # test size 0.2 values
scalers = {} # stores scaler model for general and minimal <model_name: scaler>

# Prepare predictions from models
for i in range(len(models)):
    model_name, model = models[i]
    data = pd.read_csv(training_dataset[model_name], header=None, skiprows=1)
    X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=69)
    
    if model_name in ["rf-minimal.pkl", "rf-general.pkl"]:
        scaler = MaxAbsScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        scalers[model_name] = scaler

    if testing_ensemble_y is None:
        testing_ensemble_y = y_test

    testing_ensemble_X[model_name] = X_test

### Test on 0.2 validation split initial dataset

In [45]:
y_ensemble_pred = None # holds predictions of all models
model_col = {} # Takes note of which column the model predictions are in <index, model_name>
model_col_rev = {} # <model_name, index>

for i in range(len(models)):
    model_name, model = models[i]
    X_test = testing_ensemble_X[model_name] # Already scaled
    
    y_pred = model.predict(X_test)
    if y_ensemble_pred is None:
        y_ensemble_pred = np.empty((y_pred.shape[0], 3))

    y_ensemble_pred[:, i] = y_pred
    model_col[i] = model_name
    model_col_rev[model_name] = i

# Look at priority list
pred_combined_y = y_ensemble_pred[:, model_col_rev["rf-general.pkl"]]

row = 0
for entry in y_ensemble_pred:
    index_benign_spec = model_col_rev["rf-minimal.pkl"]
    index_lexical = model_col_rev["rf-lexical.pkl"]
    if entry[index_benign_spec] == 0 and entry[index_lexical] == 0:
        pred_combined_y[row] = 0
    row += 1

/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [46]:
precision = precision_score(testing_ensemble_y, pred_combined_y, average='weighted')
recall = recall_score(testing_ensemble_y, pred_combined_y, average='weighted')
f1 = f1_score(testing_ensemble_y, pred_combined_y, average='weighted')
val_accuracy = accuracy_score(testing_ensemble_y, pred_combined_y)
print(f'OVERALL: Accuracy: {val_accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, F1 Score: {f1:.8f}')

class_test_precision, class_test_recall, class_test_f1, class_ = precision_recall_fscore_support(testing_ensemble_y, pred_combined_y)
for i in range(4):
    print(f'Class {i}:\tTest Precision: {class_test_precision[i]:.8f},\tTest Recall: {class_test_recall[i]:.8f},\tTest f1: {class_test_f1[i]:.8f}')
calc_FNR_accuracy(testing_ensemble_y, pred_combined_y)

OVERALL: Accuracy: 0.93192515, Precision: 0.93366513, Recall: 0.93192515, F1 Score: 0.92897798
Class 0:	Test Precision: 0.91870577,	Test Recall: 0.99287799,	Test f1: 0.95435289
Class 1:	Test Precision: 0.97495518,	Test Recall: 0.90272878,	Test f1: 0.93745286
Class 2:	Test Precision: 0.93784501,	Test Recall: 0.69886212,	Test f1: 0.80090609
Class 3:	Test Precision: 0.99540582,	Test Recall: 0.88717015,	Test f1: 0.93817657
Accuracy for class 0 : 0.9375379110711846
FNR for class 0 : 0.007122008172796264
Accuracy for class 1 : 0.9821712390297838
FNR for class 1 : 0.09727121809504047
Accuracy for class 2 : 0.9500610416234768
FNR for class 2 : 0.30113788129707786
Accuracy for class 3 : 0.9940801142514915
FNR for class 3 : 0.11282984531392175


### Test on phishing dataset

In [47]:
training_dataset_phishing = {'rf-minimal.pkl': 'rf-minimal-test-phishing.csv',
                    'rf-general.pkl': 'rf-general-test-phishing.csv',
                    'rf-lexical.pkl': 'rf-lexical-test-phishing.csv'}

input_ensemble_phishing = None

# Prepare predictions from models
for i in range(len(models)):
    model_name, model = models[i]
    X_test = pd.read_csv(training_dataset_phishing[model_name], header=None, skiprows=1)
    if model_name in ["rf-minimal.pkl", "rf-general.pkl"]:
        X_test = scalers[model_name].transform(X_test)

    y_pred = model.predict(X_test) # prediction for model
    if input_ensemble_phishing is None:
        input_ensemble_phishing = np.empty((y_pred.shape[0], 3))

    input_ensemble_phishing[:, i] = y_pred

/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [48]:
# Phishing is label 2.
y_phishing = np.full((input_ensemble_phishing.shape[0],), 2)
pred_combined_y_phishing = input_ensemble_phishing[:, model_col_rev["rf-general.pkl"]]

row = 0
for entry in input_ensemble_phishing:
    index_benign_spec = model_col_rev["rf-minimal.pkl"]
    index_lexical = model_col_rev["rf-lexical.pkl"]
    if entry[index_benign_spec] == 0 and entry[index_lexical] == 0:
        pred_combined_y_phishing[row] = 0
    row += 1

In [49]:
precision = precision_score(y_phishing, pred_combined_y_phishing, average='weighted')
recall = recall_score(y_phishing, pred_combined_y_phishing, average='weighted')
f1 = f1_score(y_phishing, pred_combined_y_phishing, average='weighted')
val_accuracy = accuracy_score(y_phishing, pred_combined_y_phishing)
print(f'OVERALL: Accuracy: {val_accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, F1 Score: {f1:.8f}')

class_test_precision, class_test_recall, class_test_f1, class_ = precision_recall_fscore_support(y_phishing, pred_combined_y_phishing)
for i in range(4):
    print(f'Class {i}:\tTest Precision: {class_test_precision[i]:.8f},\tTest Recall: {class_test_recall[i]:.8f},\tTest f1: {class_test_f1[i]:.8f}')
calc_FNR_accuracy(y_phishing, pred_combined_y_phishing)

OVERALL: Accuracy: 0.67870517, Precision: 1.00000000, Recall: 0.67870517, F1 Score: 0.80860556
Class 0:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 1:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 2:	Test Precision: 1.00000000,	Test Recall: 0.67870517,	Test f1: 0.80860556
Class 3:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Accuracy for class 0 : 0.7167268310785736
FNR for class 0 : -1
Accuracy for class 1 : 0.9648291218244999
FNR for class 1 : -1
Accuracy for class 2 : 0.6787051718902871
FNR for class 2 : 0.3212948281097129
Accuracy for class 3 : 0.9971492189872136
FNR for class 3 : -1


/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Test on benign dataset

In [50]:
training_dataset_benign = {'rf-minimal.pkl': 'rf-minimal-test-benign.csv',
                    'rf-general.pkl': 'rf-general-test-benign.csv',
                    'rf-lexical.pkl': 'rf-lexical-test-benign.csv'}

input_ensemble_benign = None

# Prepare predictions from models
for i in range(len(models)):
    model_name, model = models[i]
    X_test = pd.read_csv(training_dataset_benign[model_name], header=None, skiprows=1)
    if model_name in ["rf-minimal.pkl", "rf-general.pkl"]:
        X_test = scalers[model_name].transform(X_test)

    y_pred = model.predict(X_test) # prediction for model

    if input_ensemble_benign is None:
        input_ensemble_benign = np.empty((y_pred.shape[0], 3))

    input_ensemble_benign[:, i] = y_pred

/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [51]:
# Benign is label 0.
y_benign = np.full((input_ensemble_benign.shape[0],), 0)

pred_combined_y_benign = input_ensemble_benign[:, model_col_rev["rf-general.pkl"]]
row = 0
for entry in input_ensemble_benign:
    index_benign_spec = model_col_rev["rf-minimal.pkl"]
    index_lexical = model_col_rev["rf-lexical.pkl"]
    if entry[index_benign_spec] == 0 and entry[index_lexical] == 0:
        pred_combined_y_benign[row] = 0
    row += 1

In [52]:
precision = precision_score(y_benign, pred_combined_y_benign, average='weighted')
recall = recall_score(y_benign, pred_combined_y_benign, average='weighted')
f1 = f1_score(y_benign, pred_combined_y_benign, average='weighted')
val_accuracy = accuracy_score(y_benign, pred_combined_y_benign)
print(f'OVERALL: Accuracy: {val_accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, F1 Score: {f1:.8f}')

class_test_precision, class_test_recall, class_test_f1, class_ = precision_recall_fscore_support(y_benign, pred_combined_y_benign)
for i in range(4):
    print(f'Class {i}:\tTest Precision: {class_test_precision[i]:.8f},\tTest Recall: {class_test_recall[i]:.8f},\tTest f1: {class_test_f1[i]:.8f}')
calc_FNR_accuracy(y_benign, pred_combined_y_benign)

OVERALL: Accuracy: 0.59436625, Precision: 1.00000000, Recall: 0.59436625, F1 Score: 0.74558308
Class 0:	Test Precision: 1.00000000,	Test Recall: 0.59436625,	Test f1: 0.74558308
Class 1:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 2:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 3:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Accuracy for class 0 : 0.59436625421562
FNR for class 0 : 0.4056337457843801
Accuracy for class 1 : 0.7702913767072176
FNR for class 1 : -1
Accuracy for class 2 : 0.8247054127692067
FNR for class 2 : -1
Accuracy for class 3 : 0.9993694647391955
FNR for class 3 : -1


/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
